In [8]:
import os, json, io, nltk, re
from collections import Counter 
from dataclasses import dataclass
import numpy as np
import gensim.downloader as api
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine

In [12]:
#nltk.download('stopwords')
# директория, где располагается снимок википедии
enwikipath = "C:/Users/monte/_Olesya/Диплом/Основная папка/задание/enwiki20201020"

# в файле catalog.json для каждой статьи категории NLP хранится имя файла из снимка википедии
cat = json.load(open("../catalog.json", "r", encoding="utf-8"))

model = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [23]:
# выполняется поиск статьи в каталоге
def FindArticle(j, art_title):
    for x in j:
        if x['title'] == art_title:
            return x
    return None


def my_comp_distance(x):
    return x.distance

@dataclass
class dist_to_root_art:
    idx: int
    art_name: str    
    distance: float
    
@dataclass
class art_dist:
    idx: int
    art_name: str  
    distances: list[dist_to_root_art]

In [20]:
def text_to_vector(text, model):
    # Разбиваем текст на слова
    words = text.split()
    
    # Создаем список векторов для всех слов в тексте
    word_vectors = []
    for word in words:
        if word in model.key_to_index:  # Проверяем, есть ли слово в модели
            word_vectors.append(model[word])
    
    # Усредняем векторы слов для получения единого вектора текста
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Возвращаем нулевой вектор, если слов нет в модели

def calculate_distance(text1, text2, model):
    # Получаем векторы для двух текстов
    vector1 = text_to_vector(text1, model)
    vector2 = text_to_vector(text2, model)
    
    # Вычисляем косинусное расстояние между векторами
    return cosine(vector1, vector2)


In [24]:
stopwords = nltk.corpus.stopwords.words('english')
NLP_Pages = json.load(open("../NatLangProc_Pages.json", "r", encoding="utf-8"))
NLP_P = list()
for x in NLP_Pages:
    if NLP_Pages[x][0] == "Natural language processing":
        NLP_P.append(x)
        
        
        
articles = []

for i in range(len(NLP_P)):
    if (NLP_P[i] in cat):
        fil_i = cat[NLP_P[i]]
        j_i = json.load(open(enwikipath+"/"+fil_i, "r", encoding="utf-8"))
        art_i = FindArticle(j_i, NLP_P[i])
        text_i = art_i['text']
        
        
        article = art_dist(i,NLP_P[i],[])
        
        for k in range(i, len(NLP_P)):
            if (NLP_P[k] in cat):
                fil_k = cat[NLP_P[k]]
                j_k = json.load(open(enwikipath+"/"+fil_k, "r", encoding="utf-8"))
                art_k = FindArticle(j_k, NLP_P[k])
                text_k = art_k['text']
               
                
                article.distances.append(dist_to_root_art(k, NLP_P[k],calculate_distance(text1, text2, model)))
                
            progress = (len(NLP_P)*(i) + k + 1)/(len(NLP_P)**2)
            print(f"Progress: [{'='*int(progress*20)}{'>' if progress < 1 else '='}{'-'*int((1-progress)*20)}] {progress*100:3.4f}%", end = '\r')
        
        articles.append(article)

        
for i in range (len(articles)):
    for j in range(i + 1, len(articles)):
        for k in articles[i].distances:
            if k.idx == articles[j].idx:
                articles[j].distances.append(dist_to_root_art(articles[i].idx, articles[i].art_name, k.distance))
                break


In [25]:
articles_dict = {}
for i in range (len(articles)):
    idx = articles[i].idx
    art_name = articles[i].art_name
    distances = {}
    for k in articles[i].distances:
        distances[k.idx] = {"art_name": k.art_name, "distance": k.distance}
    articles_dict[idx] = {"art_name": art_name, "distances": distances}
    
with open('NLP_distances_result_word2vec.json', 'w') as file:
    json.dump(articles_dict, file)

In [37]:
def closest_art_sum (name):
    art_dict = json.load(open(path_to_folder + "/" + name + ".json", "r", encoding="utf-8"))
    sum_list = []
    for i in art_dict:
        sum_el = 0
        for j in art_dict[i]["distances"]:
            sum_el += art_dict[i]["distances"][j]["distance"]
        sum_list.append(dist_to_root_art(i, art_dict[i]["art_name"], sum_el ** (1 / 2)))
    return max(sum_list, key = my_comp_distance).art_name

def main_art (name):
    return categ[name][3]

def print_cat_main_closest (name, name_json):
    print("Category name: " + name + "\n")
    print("Main articles: ")
    list_art = ()
    list_art = main_art(name)
    for i in range(len(list_art)):
        print(list_art[i] + ", ")
    print("\n" +  "Closest article: " + closest_art_sum(name_json))
    print("-"*100)

In [38]:
categ = json.load(open("C:/Users/monte/_Olesya/Диплом/Основная папка/задание/NatLangProc_Categ.json", "r", encoding="utf-8"))
path_to_folder = "C:/Users/monte/_Olesya/Диплом/Основная папка/задание/Word2vec"
print_cat_main_closest("Natural language processing", "NLP_distances_result_word2vec")

Category name: Natural language processing

Main articles: 
Natural language processing, 

Closest article: Natural language processing
----------------------------------------------------------------------------------------------------
